In [32]:
import pandas as pd

# Импортируем датасет данного трактора, для примера положим 022C4097

In [50]:
df = pd.read_csv('/kaggle/input/tractor-dataset/df_022C4097_v2.csv', sep=',')

In [51]:
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('Дата и время', axis=1)
df = df.drop('Сост.пед.сцепл.', axis=1)
df = df.drop('Значение счетчика моточасов, час:мин', axis=1)

df['Темп.масла двиг.,°С'] = df['Темп.масла двиг.,°С'].str.replace(',', '.').astype('float')
df['Обор.двиг.,об/мин'] = df['Обор.двиг.,об/мин'].str.replace(',', '.').astype('float')
df['Полож.пед.акселер.,%'] = df['Полож.пед.акселер.,%'].str.replace(',', '.').astype('float')

In [52]:
df

,"Полож.пед.акселер.,%","Давл.масла двиг.,кПа","Темп.масла двиг.,°С","Обор.двиг.,об/мин",КПП. Температура масла,КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",...,Аварийное давление в II контуре тормозной системы (spn3855),Зарядка АКБ (spn3854),Отопитель (spn3853),Выход блока управления двигателем (spn3852),Включение тормозков (spn3859),Засоренность фильтра слива (spn3858),Аварийная температура масла ДВС(spn3856),Низкий уровень ОЖ (spn3860),Аварийная температура масла ГТР (spn3867),Подогрев топливного фильтра (spn3865)
0,0.0,380,-273.0,649.000,-273,1184,0.0,380,34,760,...,0,0,0,0,0,0,0,0,0,0
1,0.0,360,-273.0,651.000,-273,1200,0.0,360,34,816,...,0,0,0,0,0,0,0,0,0,0
2,0.0,348,-273.0,656.000,-273,1200,0.0,348,35,816,...,0,0,0,0,0,0,0,0,0,0
3,0.0,360,-273.0,671.875,-273,1200,0.0,360,36,816,...,0,0,0,0,0,0,0,0,0,0
4,0.0,328,-273.0,649.625,-273,1200,0.0,328,37,816,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18139,56.4,328,-273.0,1353.125,-273,1168,7.8,328,80,784,...,0,0,0,0,0,0,0,0,0,0
18140,29.6,220,-273.0,984.250,-273,1136,5.8,220,80,816,...,0,0,0,0,0,0,0,0,0,0
18141,31.6,224,-273.0,1015.875,-273,1136,5.9,224,80,816,...,0,0,0,0,0,0,0,0,0,0
18142,0.0,120,-273.0,648.250,-273,1040,0.0,120,80,768,...,0,0,0,0,0,0,0,0,0,0


# Используем IsolationForest для поиска аномалий

In [67]:
from sklearn.ensemble import IsolationForest
import numpy as np

model = IsolationForest(n_estimators=100, contamination=0.05)
model.fit(df)

anomaly_scores = model.decision_function(df)

threshold = np.percentile(anomaly_scores, 95)
anomalies = df[anomaly_scores > threshold]

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [68]:
anomalies

,"Полож.пед.акселер.,%","Давл.масла двиг.,кПа","Темп.масла двиг.,°С","Обор.двиг.,об/мин",КПП. Температура масла,КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",...,Аварийное давление в II контуре тормозной системы (spn3855),Зарядка АКБ (spn3854),Отопитель (spn3853),Выход блока управления двигателем (spn3852),Включение тормозков (spn3859),Засоренность фильтра слива (spn3858),Аварийная температура масла ДВС(spn3856),Низкий уровень ОЖ (spn3860),Аварийная температура масла ГТР (spn3867),Подогрев топливного фильтра (spn3865)
454,73.2,400,-273.0,1579.250,-273,1168,10.2,400,83,784,...,0,0,0,0,0,0,0,0,0,0
458,74.8,408,-273.0,1635.500,-273,1168,10.5,408,84,784,...,0,0,0,0,0,0,0,0,0,0
812,73.6,404,-273.0,1623.000,-273,1152,10.3,404,84,784,...,0,0,0,0,0,0,0,0,0,0
836,78.4,420,-273.0,1659.750,-273,1168,10.8,420,83,784,...,0,0,0,0,0,0,0,0,0,0
1653,77.2,408,-273.0,1669.250,-273,1152,10.7,408,84,784,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17977,77.6,400,-273.0,1644.625,-273,1152,10.7,400,82,784,...,0,0,0,0,0,0,0,0,0,0
17985,76.4,400,-273.0,1648.125,-273,1152,10.9,400,82,784,...,0,0,0,0,0,0,0,0,0,0
17993,77.6,408,-273.0,1644.000,-273,1152,11.0,408,82,784,...,0,0,0,0,0,0,0,0,0,0
17997,75.6,400,-273.0,1617.375,-273,1152,10.7,400,82,784,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
anomalies.to_csv('anomalies.csv')